<img src="../../shared/img/banner.svg"></img>

# Lab 03 - Models and Monsters

In [ ]:
%matplotlib inline

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed
from shared.src import style

In [ ]:
import random

from client.api.notebook import Notebook
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import scipy.stats

import shared.src.utils.util as util

In [ ]:
sns.set_context("notebook", font_scale=1.7)

In [ ]:
ok = Notebook("ok/config")

## Learning Objectives


1. Practice building models of random processes with pyMC.
1. Learn to recognize problems in real life that you can use models to solve.

## Distributions & Dragons

In [ ]:
Image("./img/dndlogo.jpg", width=500)

[*Dungeons & Dragons*](http://dnd.wizards.com/) (R), or D&D, is the world's most popular tabletop roleplaying game.
In addition to non-random, human-controlled elements of storytelling, it features lots of cases where story outcomes are determined by the rolling of dice of varying sides, most prominently a 20-sided die.

One of the most important examples is *attacking*, when one character tries to use a weapon against another.
The core idea is as follows:

1. The attacker rolls a 20 sided die (`pm.DiscreteUniform`, with `lower=1` and `upper=20`).
1. If the number on that die is as high or higher than the armor of the opponent, the attack "hits" and deals damage.
1. If the number on that die is also as high or higher than the "critical range" of the attacker's weapon, then the attacker rolls another 20 sided die. This is called a "confirmation roll".
1. If that number is also higher than the armor of the opponent, the attacker deals a "critical hit", meaning the attacker deals damage again.

A character is dead if the amount of damage dealt exceeds their health points,
so being able to do more damage to enemies is better.

Unlike in the case of games like
[*Risk*](https://en.wikipedia.org/wiki/Risk_(game)) or
[*Root*](https://boardgamegeek.com/boardgame/237182/root) it can be much harder in D&D to use math to calculate things like how much damage a weapon will do on average or in the median (especially once additional rules get added to the above!).

However, it's not that much harder to *simulate*!
The function below produces models for D&D attacks.
Ignore the `magic_bonus` argument for now.

In [ ]:
def construct_dnd_model(function_to_add_damage, magic_bonus=0):
    """
    Arguments
    ---------
    function_to_add_damage: function, takes in the attack_roll variable and adds variables for the damage rolls
    magic_bonus: integer, value to add to the attack, confirmation, and damage rolls for enchanted weapons
    
    Returns
    -------
    dnd_model: pm.Model, model for the random process of rolling attack and damage in DnD.
    """
    dnd_model = pm.Model()

    with dnd_model:

        attack_roll = pm.DiscreteUniform(name="attack_roll",
                                         lower=1, upper=20)
        confirmation_roll = pm.DiscreteUniform(name="confirmation_roll",
                                               lower=1, upper=20)

        armor = 15
        attack_hit = pm.Deterministic(name="attack_hit",
                                      var=attack_roll + magic_bonus >= armor)

        base_damage_roll, critical_damage_roll, critical_threat = function_to_add_damage(attack_roll)

        critical_confirmed = pm.Deterministic(name="critical_confirmed",
                                              var=confirmation_roll + magic_bonus >= armor)

        critical_hit = pm.Deterministic(name="critical_hit",
                                        var=attack_hit * critical_threat * critical_confirmed)

        total_damage = pm.Deterministic(name="total_damage",
                                        var=attack_hit * (base_damage_roll + magic_bonus)
                                        + critical_hit * (critical_damage_roll + magic_bonus))
        
    return dnd_model

Note that the simulation above is not an exact step-for-step implementation of the process described by the rules.
For example, instead of, e.g., only doing the `confirmation_roll` when the `attack_roll` is above the `critical_range`, we always do it, but we only add in the damage if `critical_hit` is `True`.

Because different weapons do damage differently, we need to provide a `function_to_add_damage` to complete our model.
The template below shows the basic outline of such a function.

```python
def function_to_add_damage(attack_roll):
    """This is a format for functions to pass to construct_dnd_model.
    
    Exact content differs based on the weapon involved, but the four variables
    critical_range, base_damage_roll, critical_damage_roll, and critical_threat
    are always involved, and the latter three are returned.
    
    Arguments
    ---------
    attack_roll: pyMC Variable, random variable representing the attack roll
    
    Returns
    ---------
    base_damage_roll: pyMC Variable, random variable representing damage with no critical hit
    critical_damage_roll: pyMC Variable, random variable representing possible additional critical hit damage
    critical_threat: pyMC Variable, random variable representing whether attack roll at least
                     critical_range, either 0 or 1
    """
    critical_range = ?
    
    base_damage_roll = ?
    critical_damage_roll = ?
    
    critical_threat = pm.Deterministic(name="critical_threat",
                                       var=attack_roll >= critical_range)
    
    return base_damage_roll, critical_damage_roll, critical_threat
```

An example weapon is below.

In [ ]:
def add_greatsword(attack_roll):
    
    critical_range = 19
    
    base_damage_roll = pm.Deterministic(name="base_damage_roll",
                                        var=pm.DiscreteUniform("bd_d6_1", lower=1, upper=6) +
                                        pm.DiscreteUniform("bd_d6_2", lower=1, upper=6))
    
    critical_damage_roll = pm.Deterministic(name="critical_damage_roll",
                                            var=pm.DiscreteUniform("cd_d6_1", lower=1, upper=6) +
                                            pm.DiscreteUniform("cd_d6_2", lower=1, upper=6))
    
    critical_threat = pm.Deterministic(name="critical_threat",
                                       var=attack_roll >= critical_range)
    
    return base_damage_roll, critical_damage_roll, critical_threat

In D&D terms, the greatsword would be described as dealing damage with _two six-sided die_
(notice the `base_damage_roll` adds together two `DiscreteUniform` random variables between `1` and `6`)
and having a _critical range of 19_.

The code cells below will construct a model for attacking with the greatsword, draw 1000 samples from it, and then convert those samples into a dataframe for ease of analysis.

In [ ]:
greatsword_model = construct_dnd_model(add_greatsword)

In [ ]:
with greatsword_model:
    greatsword_attacks = pm.sample(draws=1000, chains=1)

In [ ]:
greatsword_df = util.samples_to_dataframe(greatsword_attacks)

From the samples, compute the following:

- `gs_hit_chance:` Fraction of times a hit happens
- `gs_avg_dmg`: Average damage, including misses
- `gs_avg_dmg_crit`: Average damage on a critical hit
- `gs_std_dmg`: Standard deviation of the damage

In [ ]:
ok.grade("q1")

### Do You Believe in Magic?

Some weapons also provide a `magic_bonus`, which is added to the attack roll, the confirmation roll, and the damage (see `construct_dnd_model` for details).
Traditionally, a weapon with name _weaponname_ and a `magic_bonus=K` is called a *weaponname +K*.

In some cells below, use `construct_dnd_model` to build a model, `greatsword_plusone_model`,
for a *greatsword +1*.
You can still use `add_greatsword`;
you only need to change the `magic_bonus` argument to `construct_dnd_model`.

Then draw 1000 samples, put them into a dataframe,
and compute the statistics.
It will look very much like the code above.

Give the computed statistics the same variable names,
but with `gs1` as the prefix, instead of `gs`.

In [ ]:
ok.grade("q2")

In [ ]:
ok.score()

### A Critical Decision

As a reward for completing a quest, a wizard (he/him) offers your friends,
Cora the Paladin (she/her) and Baab the Goblin (they/them), a choice of two weapons:
a *keen [falchion]((https://en.wikipedia.org/wiki/Falchion)) +1*
and a *dagger +3*. There's two of them and two weapons, so they'll need to each take one.

Cora wants the weapon that's the most dependable and does damage most consistently,
while Baab wants the weapon that's more likely to make a big splash and do more damage when it hits.
As the party's datamancer, it is your responsibility to provide data-driven modeling services and support optimal adventuring decision-making.

Help Cora and Baab divide up the weapons by simulating the damage output for both of them.
Produce samples, visualize the sampling distributions of their statistics,
and then provide a recommendation for who should get which weapon.

Here's how you'll do it:

Write two functions below which add weapon damage rolls and critical threats
to the model:
`add_keen_falchion` and
`add_dagger`.

The *keen falchion +1* has a `critical_range` of `15` and rolls two four-sided die. It has a `magic_bonus` of `1`.
The *dagger +3* has a `critical_range` of `19` and rolls two four-sided die. It has `magic_bonus=3`.

Give those functions and the magic bonus values to `construct_dnd_model` separately
to make two `pyMC.Model`s, one for the `keen_falchion` and the other for the `dagger`.
To start, draw a thousand of samples from each.

Create visualizations for the resulting data,
comparing the `total_damage` of the *dagger +3* to that of the *keen falchion +1*.
Calculate descriptive statistics on the damage, including at least the mean, skew, and the chance to hit.
(Hint: compute the skew with the `.skew` method or with `scipy.stats.skew`).

Use bootstrapping to represent and visualize your uncertainty about those parameters.
You might also be interested in the median, the 1st and 3rd quartiles, the chance of a critical hit,
or the 90th percentile of the damage, given that the weapon hits,
which can help make your argument to Cora and Baab stronger.

Once all of your code is working, change the number of samples to 10000.
Combine your visualizations and your statistics into a report to Cora and Baab about the two weapons.
End your report with your personal recommendation.
If you feel neither party member has a strong reason to prefer either, you're free to claim that as well!
Just make your argument data-driven -- you are a _datamancer_, not an emotion lord.

#### Q Write your report to Cora and Baab here. Connect your claims to your plots and to your computed statistics.